In [97]:
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [98]:

# Load the IMDB dataset word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [99]:
model = load_model('simple_rnn_imdb.h5')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [100]:
model.get_weights()

[array([[-0.3557817 ,  0.26718327, -0.2841122 , ...,  0.31970286,
         -0.48272935, -0.30368188],
        [ 0.04548381, -0.00371143,  0.03475645, ...,  0.04194489,
          0.05458818, -0.01742488],
        [ 0.08171488, -0.02424334,  0.03564453, ..., -0.10258467,
         -0.02065501,  0.02457915],
        ...,
        [-0.00668605, -0.00669285, -0.06938586, ...,  0.01400633,
          0.02566169, -0.04927238],
        [ 0.04046777, -0.02974709, -0.02908854, ...,  0.07081588,
          0.06154044, -0.01328787],
        [ 0.02846931, -0.05086194,  0.01577872, ..., -0.08869761,
          0.08163014,  0.04669416]], dtype=float32),
 array([[-0.08746209,  0.13788089,  0.1395356 , ...,  0.14516857,
          0.06409428,  0.06455675],
        [-0.08925038,  0.08685303, -0.01050265, ..., -0.15092848,
          0.08376821, -0.09371262],
        [-0.08084508,  0.00230656, -0.11913824, ...,  0.15817182,
         -0.12985289,  0.05917152],
        ...,
        [-0.0454497 ,  0.11414427, -0.1

In [101]:
# Step 2: Helper Functions
# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input
def preprocess_text(text):
    # Add better text cleaning
    import re
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation
    text = text.lower().strip()
    
    words = text.split()
    # Handle unknown words better (use index 2 for unknown, not word_index.get(word, 2))
    encoded_review = [word_index.get(word, 2) for word in words]
    encoded_review = [x + 3 for x in encoded_review]  # Adjust indexing
    
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

In [102]:
def predict_sentiment(review):
    processed = preprocess_text(review)
    prediction = model.predict(processed)
    sentiment = 'Positive' if prediction[0][0] > 0.3 else 'Negative'
    return sentiment, prediction[0][0]


In [103]:
# Step 4: User Input and Prediction
# Example review for prediction
example_review = "This movie was fantastic! The acting was great and the plot was thrilling."
sentiment,score=predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step
Review: This movie was fantastic! The acting was great and the plot was thrilling.
Sentiment: Negative
Prediction Score: 0.020746205002069473
